In [63]:
import soundfile as sf
import torch
from datasets import load_dataset
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor, Wav2Vec2FeatureExtractor, Wav2Vec2CTCTokenizer

In [3]:
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

Downloading:   0%|          | 0.00/159 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/291 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/163 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.60k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/378M [00:00<?, ?B/s]

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [36]:
librispeech_samples_ds = load_dataset("patrickvonplaten/librispeech_asr_dummy", "clean", split="validation")

# load audio
audio_input, sample_rate = sf.read(librispeech_samples_ds[2]["file"])

# pad input values and return pt tensor
input_values = processor(audio_input, sampling_rate=sample_rate, return_tensors="pt").input_values

# INFERENCE

# retrieve logits & take argmax
logits = model(input_values).logits
predicted_ids = torch.argmax(logits, dim=-1)

# transcribe
transcription = processor.decode(predicted_ids[0])

Reusing dataset librispeech_asr (/home/slzhang/.cache/huggingface/datasets/librispeech_asr/clean/2.1.0/468ec03677f46a8714ac6b5b64dba02d246a228d92cbbad7f3dc190fa039eab1)


In [41]:
# FINE-TUNE

target_transcription = "A MAN SAID TO THE UNIVERSE"

# encode labels
with processor.as_target_processor():
  labels = processor(target_transcription, return_tensors="pt").input_ids

In [52]:
predicted_ids.shape

torch.Size([1, 624])

In [53]:
input_values.shape

torch.Size([1, 199760])

In [77]:
extractor = Wav2Vec2FeatureExtractor.from_pretrained("facebook/wav2vec2-base-960h")
decoder = Wav2Vec2CTCTokenizer
input_values = extractor(audio_input, sampling_rate=sample_rate, return_tensors="pt").input_values
logits = model(input_values).logits
predicted_ids = torch.argmax(logits, dim=-1)
transcription = decoder.decode(predicted_ids[0])
transcription


TypeError: decode() missing 1 required positional argument: 'token_ids'

In [70]:
predicted_ids[0].shape

torch.Size([624])

In [75]:
dummy_input = torch.randn(500)
torch.onnx.export(decoder.decode(), dummy_input, "./Wav2Vec_decoder.onnx")

TypeError: decode() missing 1 required positional argument: 'token_ids'

In [62]:
for k,v in model.named_parameters():
    print(k)

wav2vec2.masked_spec_embed
wav2vec2.feature_extractor.conv_layers.0.conv.weight
wav2vec2.feature_extractor.conv_layers.0.layer_norm.weight
wav2vec2.feature_extractor.conv_layers.0.layer_norm.bias
wav2vec2.feature_extractor.conv_layers.1.conv.weight
wav2vec2.feature_extractor.conv_layers.2.conv.weight
wav2vec2.feature_extractor.conv_layers.3.conv.weight
wav2vec2.feature_extractor.conv_layers.4.conv.weight
wav2vec2.feature_extractor.conv_layers.5.conv.weight
wav2vec2.feature_extractor.conv_layers.6.conv.weight
wav2vec2.feature_projection.layer_norm.weight
wav2vec2.feature_projection.layer_norm.bias
wav2vec2.feature_projection.projection.weight
wav2vec2.feature_projection.projection.bias
wav2vec2.encoder.pos_conv_embed.conv.bias
wav2vec2.encoder.pos_conv_embed.conv.weight_g
wav2vec2.encoder.pos_conv_embed.conv.weight_v
wav2vec2.encoder.layer_norm.weight
wav2vec2.encoder.layer_norm.bias
wav2vec2.encoder.layers.0.attention.k_proj.weight
wav2vec2.encoder.layers.0.attention.k_proj.bias
wav2ve